# Contents : 

 - Implementing a Pipeline to Process Documents and create a vocabulary specific to the data considered
 - Using Pretrained Glove embeddings for faster lookup
 - Implementing hash tables using Locality Sensitive Hashing
 - Retrieved documents with simlar embeddings using LSH and the lookup architecture developed

In [141]:
import numpy as np
import pandas as pd
import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.stem import PorterStemmer

import gensim
import scipy
import sklearn
from gensim.models import KeyedVectors

In [62]:
def get_dict(file_name):
    """
    This function returns the english to french dictionary given a file where the each column corresponds to a word.
    Check out the files this function takes in your workspace.
    """
    my_file = pd.read_csv(file_name, delimiter=' ')
    etof = {}  # the english to french dictionary to be returned
    for i in range(len(my_file)):
        # indexing into the rows.
        en = my_file.loc[i][0]
        fr = my_file.loc[i][1]
        etof[en] = fr

    return etof


In [63]:
def cosine_similarity(A, B):
    '''
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        cos: numerical number representing the cosine similarity between A and B.
    '''
    # you have to set this variable to the true label.
    cos = -10
    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)

    return cos


In [19]:
def load_data():
    data = pd.read_csv('review.csv')
    data = data[['reviews.text','reviews.title']]
    data['Title_Review'] = data['reviews.text']+' ' + data['reviews.title']
    data = data.iloc[:,-1]
    return data

In [47]:
def clean_data(data):
    tokens = data.split()
    
    re_punct = re.compile('[%s]'%re.escape(string.punctuation))
    
    cleaned_text = [re_punct.sub('',w) for w in tokens]
    
    cleaned_text = [word for word in cleaned_text if word.isalpha()]
    
    stop_words =  stopwords.words('english')
    cleaned_text = [word for word in cleaned_text if word not in stop_words]
    
    cleaned_text = [word for word in cleaned_text if len(word) > 1]
    
    return cleaned_text

In [54]:
def add_doc_to_vocab(vocabulary):
    data = load_data()
    for item in list(data):
        clean_text = clean_data(str(item))
        vocabulary.update(clean_text)

In [55]:
def process_vocabulary(min_occurences,vocabulary):
    tokens = [word for word,count in vocabulary.items() if count >= min_occurences]
    return tokens

In [56]:
def save_list(word_list, file_name):
    string = '\n'.join(word_list)
    file = open(file_name,'w')
    file.write(string)
    file.close()


In [57]:
vocabulary = Counter()
min_occurences = 2
add_doc_to_vocab(vocabulary)
tokens = process_vocabulary(min_occurences,vocabulary)
save_list(tokens, 'vocabulary.txt')

#### >>> Checkpoint :  At this point we have a vocabulary of all the words present in the reviews

In [64]:
review_len = [len(str(item).split()) for item in load_data()]
max_len = max(review_len)
max_len

/Users/shekhartanwar/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


1868

In [65]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel

In [66]:
File = 'glove.6B.300d.txt'
gloveModel = loadGloveModel(File)

Loading Glove Model
400000  words loaded!


In [121]:
def process_doc(data):
    
    data = str(data)
    cleaned_text = re.sub(r'\$\w*', '', data)
    # remove old style retweet text "RT"
    cleaned_text = re.sub(r'^RT[\s]+', '', data)
    # remove hyperlinks
    cleaned_text = re.sub(r'https?:\/\/.*[\r\n]*', '', data)
    # remove hashtags
    # only removing the hash # sign from the word
    cleaned_text = re.sub(r'#', '', data)
    
    tokens = data.split()
    
    re_punct = re.compile('[%s]'%re.escape(string.punctuation))
    
    cleaned_text = [re_punct.sub('',w) for w in tokens]
    
    
    cleaned_text = [word for word in cleaned_text if word.isalpha()]
    

    cleaned_text = [word for word in cleaned_text if len(word) > 1]
    
    
    return cleaned_text

In [122]:
def get_document_embedding(doc, gloveModel):
    
    word_list = process_doc(doc)
    
    document_embedding = np.zeros(300)
    
    for word in word_list:
        document_embedding = document_embedding + gloveModel.get(word,0)
    return document_embedding

### Testing for One Document

In [123]:
doc = "Hey! how have you been?"
doc_embedding = get_document_embedding(doc, gloveModel)
doc_embedding

array([-8.0332600e-01,  7.5041400e-01, -7.1783900e-02, -1.9265000e-02,
        4.7959300e-01,  6.0707500e-01, -8.1272000e-02,  6.5340000e-02,
        2.3001400e-01, -8.5065000e+00,  1.2674200e+00, -6.1185700e-01,
       -4.1832900e-01,  5.1154200e-01, -4.2860260e-01,  2.0099520e-01,
       -5.2921300e-01, -2.7080200e-01, -4.2990000e-01, -1.6857600e-01,
        3.3753000e-01,  2.2448900e+00,  1.0648700e+00, -2.2478900e-01,
       -1.5229500e+00,  2.6710000e-01, -3.7438000e-02, -1.3987400e+00,
       -3.9200800e-01,  9.8495200e-01,  6.3252100e-01,  1.5341100e+00,
       -1.4177600e+00, -7.5303900e-01, -4.0441400e+00,  1.8751000e-01,
       -3.1893250e-01,  3.1450600e-01,  9.9340000e-02, -5.7609200e-01,
        1.0635300e-01, -1.2342800e+00, -5.9192700e-01,  3.3345500e-01,
       -6.9512000e-02,  7.5312100e-01,  2.9538100e-01,  4.6013000e-01,
       -7.2189000e-01,  8.4915900e-01,  8.3845000e-01, -1.4046400e+00,
        1.7329400e-01, -1.2066000e-01, -8.5324900e-01,  1.3768000e+00,
      

### Applying for All Docs

 - Store All Document Vectors into a Dictionary at respective indices
 - Create an array having all document vectors stacked to make a 2D array

In [130]:
def get_document_vecs(all_docs, gloveModel):
    
    index_to_DocVector = {}
    document_vectors = []
    
    for index, doc in enumerate(all_docs):
        
        
        document_embedding = get_document_embedding(doc, gloveModel)
        
        document_vectors.append(document_embedding)
        
        index_to_DocVector[index] = document_embedding
    
    
    document_embedding_matrix = np.vstack(document_vectors)
    
    return document_embedding_matrix, index_to_DocVector

In [131]:
all_document_embeddings, index_to_DocVector = get_document_vecs(load_data(), gloveModel)

/Users/shekhartanwar/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [133]:
print(f"length of dictionary {len(index_to_DocVector)}")
print(f"shape of document_vecs {all_document_embeddings.shape}")

length of dictionary 34660
shape of document_vecs (34660, 300)


### Document Lookup

In [129]:
doc = "I really liked the product. My kids use it all the time!"

doc_embedding = get_document_embedding(doc, gloveModel)

In [140]:
np.argmax((cosine_similarity(all_document_embeddings, doc_embedding)))

21231

In [136]:
# Now we look for a review that is similar to our review
data = load_data()
similar_document_index = np.argmax(cosine_similarity(all_document_embeddings, doc_embedding))
similar_document_index

21231

In [138]:
data[similar_document_index]

"This is a review of the Kindle Paperwhite launched July 2015. Essentially. the same as the previous Kindle Paperwhite but with a fantastic upgraded (300 dpi) screen, more memory and storage it's a terrific reading device. This review aims to describe both this product, and compare it with the other Kindle devices available to help you choose the best one for you.Before I start, Ive noticed several complains about Kindle not supporting the ePub format used by Canadian public libraries. Dont be put off, theres a solution in the Frequently Asked Questions section below.This review is broken up into sections so you can skip the less interesting bits. If youre in a hurry Ive included a summary at the start, and a Frequently Asked Questions section at the end.Summary - What can you do with it-----------------------------------------* Read a book. It can store 1,000s of books - take an entire library away with you on holiday.* Buy a book. You can browse the Amazon kindle store, and buy books

### Observation : 

Even though the returned text is too long, however if you copy the text on a text pad say sublime or any other software, you will find all the words in the original document in the returned document

# Locality Sensitive Hashing

We will improve on the lookup feature developed. Instead of scanning through the 34660 reviews, we will look at a subset of reviews, this will accompalish two tasks : 

1) Returned Reviews will be most similar ( based on LSH )


2) Since only a subset of reviews will be focussed on, this will speed up the retrieval time and will extract only the nearest neighbour to the review we are interested in

In [142]:
N_VECS = len(data)       # This many vectors.
N_DIMS = len(index_to_DocVector[1])     # Vector dimensionality.
print(f"Number of vectors is {N_VECS} and each has {N_DIMS} dimensions.")

Number of vectors is 34660 and each has 300 dimensions.


#### Choosing the number of planes

* Each plane divides the space to $2$ parts.
* So $n$ planes divide the space into $2^{n}$ hash buckets.
* We want to organize 34,600 document vectors into buckets so that every bucket has about $~16$ vectors.
* For that we need $\frac{34600}{16}=2163$ buckets.
* We're interested in $n$, number of planes, so that $2^{n}= 2163$. Now, we can calculate $n=\log_{2}2163 = 11.07 \approx 11$.

In [144]:
# The number of planes. We use log2(625) to have ~16 vectors/bucket.
N_PLANES = 11
# Number of times to repeat the hashing to improve the search.
N_UNIVERSES = 25

## Getting the hash number for a vector

For each vector, we need to get a unique number associated to that vector in order to assign it to a "hash bucket".

### Hyperlanes in vector spaces
* In $3$-dimensional vector space, the hyperplane is a regular plane. In $2$ dimensional vector space, the hyperplane is a line.
* Generally, the hyperplane is subspace which has dimension $1$ lower than the original vector space has.
* A hyperplane is uniquely defined by its normal vector.
* Normal vector $n$ of the plane $\pi$ is the vector to which all vectors in the plane $\pi$ are orthogonal (perpendicular in $3$ dimensional case).

### Using Hyperplanes to split the vector space
We can use a hyperplane to split the vector space into $2$ parts.
* All vectors whose dot product with a plane's normal vector is positive are on one side of the plane.
* All vectors whose dot product with the plane's normal vector is negative are on the other side of the plane.

### Encoding hash buckets
* For a vector, we can take its dot product with all the planes, then encode this information to assign the vector to a single hash bucket.
* When the vector is pointing to the opposite side of the hyperplane than normal, encode it by 0.
* Otherwise, if the vector is on the same side as the normal vector, encode it by 1.
* If you calculate the dot product with each plane in the same order for every vector, you've encoded each vector's unique hash ID as a binary number, like [0, 1, 1, ... 0].

The hash table `hashes` is list of `N_UNIVERSES` matrices, each describes its own hash table. Each matrix has `N_DIMS` rows and `N_PLANES` columns. Every column of that matrix is a `N_DIMS`-dimensional normal vector for each of `N_PLANES` hyperplanes which are used for creating buckets of the particular hash table.


We will complete the function `hash_value_of_vector` which places vector `v` in the correct hash bucket.

* We will First multiply the vector `v`, with a corresponding plane. This will give us a vector of dimension $(1,\text{N_planes})$.
* We will then convert every element in that vector to 0 or 1.
* We create a hash vector by doing the following: if the element is negative, it becomes a 0, otherwise you change it to a 1.
* We then compute the unique number for the vector by iterating over `N_PLANES`
* Then We multiply $2^i$ times the corresponding bit (0 or 1).
* We will then store that sum in the variable `hash_value`.

**Intructions:** Create a hash for the vector in the function below.
Use this formula:

$$ hash = \sum_{i=0}^{N-1} \left( 2^{i} \times h_{i} \right) $$

#### Create the sets of planes
* Create multiple (25) sets of planes (the planes that divide up the region).
* You can think of these as 25 separate ways of dividing up the vector space with a different set of planes.
* Each element of this list contains a matrix with 300 rows (the word vector have 300 dimensions), and 11 columns (there are 11 planes in each "universe").

In [145]:
def create_planes():
    np.random.seed(0)
    planes_l = [np.random.normal(size=(N_DIMS, N_PLANES))
            for _ in range(N_UNIVERSES)]
    return planes_l

In [151]:
def hash_value_of_vector(v, planes):
    
    dot_product = np.dot(v,planes)
    
    sign_of_dot_product = np.sign(dot_product)
    
    h = sign_of_dot_product > 0
    
    h = np.squeeze(h)
    
    hash_value = 0
    
    number_planes = planes.shape[1]
    
    for i in range(number_planes):
        hash_value = hash_value + np.power(2,i) * h[i]
    
    hash_value = int(hash_value)

    return hash_value

    

In [152]:
np.random.seed(0)
idx = 0
planes_l = create_planes()
planes = planes_l[idx]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
print(f" The hash value for this vector,",
      f"and the set of planes at index {idx},",
      f"is {hash_value_of_vector(vec, planes)}")

 The hash value for this vector, and the set of planes at index 0, is 512


# Hash Table

![alt text](LSH.png "Title")

In [158]:
def make_hash_table(document_vecs, planes):
    
    number_of_planes = planes.shape[1]
    
    number_buckets = 2**number_of_planes
    
    hash_table = {i:[] for i in range(number_buckets)}
    
    index_table = {i:[] for i in range(number_buckets)}
    
    for i,v in enumerate(document_vecs):
        
        hash_value = hash_value_of_vector(v, planes)
        
        
        hash_table[hash_value].append(v)
        
        index_table[hash_value].append(i)
        
        
    return hash_table, index_table
        
    

### Testing For One Plane in 0th Universe

In [159]:
np.random.seed(0)
planes = planes_l[0]  # get one 'universe' of planes to test the function
vec = np.random.rand(1, 300)
tmp_hash_table, tmp_id_table = make_hash_table(all_document_embeddings, planes)

print(f"The hash table at key 0 has {len(tmp_hash_table[0])} document vectors")
print(f"The id table at key 0 has {len(tmp_id_table[0])}")
print(f"The first 5 document indices stored at key 0 of are {tmp_id_table[0][0:5]}")

The hash table at key 0 has 162 document vectors
The id table at key 0 has 162
The first 5 document indices stored at key 0 of are [309, 528, 668, 783, 1007]


### Creating Planes in All Universes

In [164]:
def get_all_hash_index_tables(N_UNIVERSES):
    

    hash_tables, id_tables = [],[]

    for i in range(N_UNIVERSES):
        print('working on hash universe #:', i)
        planes = planes_l[i]
        hash_table, id_table = make_hash_table(all_document_embeddings, planes)

        hash_tables.append(hash_table)

        id_tables.append(id_table)

    return hash_tables,id_tables
    

In [165]:
hash_tables,id_tables = get_all_hash_index_tables(N_UNIVERSES)

working on hash universe #: 0
working on hash universe #: 1
working on hash universe #: 2
working on hash universe #: 3
working on hash universe #: 4
working on hash universe #: 5
working on hash universe #: 6
working on hash universe #: 7
working on hash universe #: 8
working on hash universe #: 9
working on hash universe #: 10
working on hash universe #: 11
working on hash universe #: 12
working on hash universe #: 13
working on hash universe #: 14
working on hash universe #: 15
working on hash universe #: 16
working on hash universe #: 17
working on hash universe #: 18
working on hash universe #: 19
working on hash universe #: 20
working on hash universe #: 21
working on hash universe #: 22
working on hash universe #: 23
working on hash universe #: 24


In [190]:
def nearest_neighbor(vector, document_vectors, K):
    
    
    # vector here is the vector we are interested in
    
    similarity_l = []
    for v in document_vectors:
        
        cosine_similarity_value = cosine_similarity(vector, v)
        similarity_l.append(cosine_similarity_value)
        
    sorted_indexes = np.argsort(similarity_l)
    
    k_th_index = sorted_indexes[-K:]
    
    return k_th_index
        

In [191]:
def approximate_KNN(search_vector_document_id , search_vector, K, planes_l, number_universes, hash_tables,id_tables):
    
    
    """
    
    INPUT :- 
        
    search_vector_document_id :  The vector which we are looking for, this is the document id for that
    search_vector             :  The vector's embedding
    K                         :  The number of neighbours we are interested in (user defined)
    planes_l                  :  All the planes which correspond to a particular universe
    number_universes          :  All the universes which we are using (here 25)
    hash_tables               :  These are all the tables present in the universes, for each sepcific universe there 
                                 would be a specific hash table
    id_tables                 :  These are all the tables which contains the id number of the document present in the
                                 dataset
    
    """
    
    
    
    
    vectors_to_consider = []
    
    indexes_to_consider = []
    
    indexes_to_consider_set = set()
    
    
    for universe_id in range(number_universes):
        
        # get all the planes in that universe
        planes = planes_l[universe_id]
        
        
        # in those planes, get the hash value of the search_vector
        hash_value = hash_value_of_vector(search_vector, planes)
        
        # get all the hash tables in that universe
        hash_table = hash_tables[universe_id]
        
        # In those hash tables look for all the document vectors which had the hash_value calculated above
        document_vectors_at_hash_value = hash_table[hash_value]
        
        
        # get all index tables in that universe
        index_table = id_tables[universe_id]
        
        # get indexes of all the documents in that index table
        document_indexes_to_consider = index_table[hash_value]
        
        if search_vector_document_id in document_indexes_to_consider:
            document_indexes_to_consider.remove(search_vector_document_id)
            print(f"removed doc_id {search_vector_document_id} of input vector from document_indexes_to_consider")
        
        for i,id_number in enumerate(document_indexes_to_consider):
            
            
            if id_number not in indexes_to_consider_set:
                
                document_vector_at_id_number = document_vectors_at_hash_value[i]
                
                vectors_to_consider.append(document_vector_at_id_number)
                
                indexes_to_consider.append(id_number)
                
                
                indexes_to_consider_set.add(id_number)
        
        # Now run k-NN on the smaller set of vecs-to-consider.
        print("Fast considering %d vecs" % len(vectors_to_consider))
        
        # convert the vecs to consider set to a list, then to a numpy array
        vecs_to_consider_arr = np.array(vectors_to_consider)
        
        # call nearest neighbors on the reduced list of candidate vectors
        nearest_neighbor_idx_l = nearest_neighbor(search_vector, vecs_to_consider_arr, K)
        
        
        # Use the nearest neighbor index list as indices into the ids to consider
        # create a list of nearest neighbors by the document ids
        nearest_neighbor_ids = [indexes_to_consider[idx]
                                for idx in nearest_neighbor_idx_l]

        return nearest_neighbor_ids


### Testing On 1 document

In [195]:
doc_id = 0

doc_to_search = data[doc_id]
vec_to_search = all_document_embeddings[doc_id]

In [196]:
K = 3
nearest_neighbor_ids  = approximate_KNN(doc_id , vec_to_search, K, planes_l, N_UNIVERSES, hash_tables,id_tables)

Fast considering 8643 vecs


In [197]:
print(f"Nearest neighbors for document {doc_id}")
print(f"Document contents: {doc_to_search}")
print("")

for neighbor_id in nearest_neighbor_ids:
    print(f"Nearest neighbor at document id {neighbor_id}")
    print(f"document contents: {data[neighbor_id]}")

Nearest neighbors for document 0
Document contents: This product so far has not disappointed. My children love to use it and I like the ability to monitor control what content they see with ease. Kindle

Nearest neighbor at document id 15478
document contents: This item works great and fitst anywhere. I have no complaints... Good buy
Nearest neighbor at document id 29732
document contents: I have the stick also much better than the stick one... Great buy
Nearest neighbor at document id 3
document contents: I've had my Fire HD 8 two weeks now and I love it. This tablet is a great value.We are Prime Members and that is where this tablet SHINES. I love being able to easily access all of the Prime content as well as movies you can download and watch laterThis has a 1280/800 screen which has some really nice look to it its nice and crisp and very bright infact it is brighter then the ipad pro costing $900 base model. The build on this fire is INSANELY AWESOME running at only 7.7mm thick and

# Conclusion
Congratulations - Now you can look up vectors that are similar to the encoding of your document using LSH!